# 🖼️ Generate Screen Descriptions
This notebook runs a selected Vision-Language Model (VLM) on 100 annotated Dutch desktop screenshots using a chosen prompt, generating structured screen descriptions for blind users.

# 🛠️ Configuration

In [1]:
MODEL_NAME = "meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8"
PROMPT_VERSION = "saliency"

PROMPT_TEXT = """
Beschrijf deze schermafbeelding voor een blinde gebruiker, met nadruk op de belangrijkste en functioneel relevante onderdelen.

Begin met het type scherm en de applicatie of website (indien zichtbaar). Beschrijf vervolgens het hoofddoel van het scherm en de belangrijkste elementen die een gebruiker nodig heeft om ermee te werken: koppen, knoppen, formulieren, foutmeldingen of statusinformatie.

Vermijd irrelevante of decoratieve details. Noem alleen elementen die bijdragen aan begrip of interactie met het scherm. Houd de beschrijving compact, neutraal en volledig in het Nederlands.
"""


# 📦 Step 1: Setup and Imports

In [2]:
import os
import base64
import json
import pandas as pd
from pathlib import Path
from datetime import datetime
from dotenv import load_dotenv
from together import Together
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, spearmanr

load_dotenv()
client = Together()

# 📂 Step 2: Load Annotations

In [ ]:
with open("../annotation_dataset/annotations.json", "r", encoding="utf-8") as f:
    annotations = json.load(f)

# 🧠 Step 3: Define Encoding and Query Functions

In [ ]:
def encode_image_to_data_url(image_path):
    """
    Convert a local PNG file to a base64-encoded data URL.
    This format is required by the Together API for vision inputs.
    """
    with open(image_path, "rb") as f:
        b64 = base64.b64encode(f.read()).decode("utf-8")
    return f"data:image/png;base64,{b64}"

def query_model(image_data_url, prompt):
    """
    Send a prompt and encoded image to the selected model on Together.
    Returns the model-generated screen description.
    """
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {"type": "image_url", "image_url": {"url": image_data_url}}
            ]
        }]
    )
    return response.choices[0].message.content.strip()

# 📸 Step 4: Evaluate One Image

In [ ]:
def evaluate_image_entry(entry, prompt, prompt_version):
    """
    Processes one image entry:
    - Loads the image by ID
    - Encodes it for Together API
    - Sends it with the prompt
    - Returns a structured result dict
    """
    image_id = entry["image_id"]
    image_path = Path(f"../annotation_dataset/images/{image_id}.png")

    if not image_path.exists():
        print(f"⚠️ Skipping {image_id} — image not found at {image_path}")
        return None

    print(f"🔍 Analyzing image {image_id}...")

    data_url = encode_image_to_data_url(image_path)
    model_output = query_model(data_url, prompt)

    print(f"✅ Done: {image_id}")
    print(f"📝 Output: {model_output.strip()}\n")

    return {
        "image_id": image_id,
        "prompt_version": prompt_version,
        "prompt": prompt,
        "model": MODEL_NAME,
        "model_output": model_output.strip()
    }

# 🚀 Step 5: Run Inference on All Screens

In [ ]:
results = []
for i, entry in enumerate(annotations):
    result = evaluate_image_entry(entry, PROMPT_TEXT, PROMPT_VERSION)
    if result:
        results.append(result)

df_results = pd.DataFrame(results)


🔍 Running image analysis 001
✅ Finished image analysis 001
Model output: Het scherm toont de startpagina van de website algoritmes.overheid.nl. Deze website is het algoritmeregister van de Nederlandse overheid.

Het hoofddoel van dit scherm is om gebruikers in staat te stellen algoritmes te zoeken, te bekijken en te registreren. 

De belangrijkste elementen op dit scherm zijn:
- Een zoekveld met de tekst "Vind een van de 822 algoritmes" en een zoekknop met de tekst "Zoeken".
- Een sectie met de titel "Bekijk de 3 meest recent gewijzigde algoritmebeschrijvingen", met drie recente algoritmebeschrijvingen.
- Een sectie met de titel "Over dit register", die informatie geeft over het doel van het algoritmeregister.
- Een sectie met de titel "Over algoritmes", die een korte uitleg geeft over wat algoritmes zijn.
- Een sectie met de titel "Samen doorontwikkelen", die informatie geeft over hoe het register open wordt doorontwikkeld.
- Een sectie met de titel "Wat kan ik hier doen?", die veelge

# 💾 Step 6: Save Results to JSON

In [ ]:
import os

results_dir = "results"
overview_dir = "results_overview"
os.makedirs(results_dir, exist_ok=True)
os.makedirs(overview_dir, exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
file_basename = f"{MODEL_NAME}_{PROMPT_VERSION}_{timestamp}"
filepath = os.path.join(results_dir, f"{file_basename}.xlsx")
os.makedirs(os.path.dirname(filepath), exist_ok=True)
df_results.to_excel(filepath, index=False)
